In [1]:
import json
import pandas as pd
import os
from pathlib import Path
import sys

sys.path.append("../")

from config import DATA_DIR
from graph_types.graph import Graph

graph = Graph.load("mag")

In [41]:
logs_dir = DATA_DIR / f"experiments/{graph.name}/subgraph_explorer"
json_files = sorted([f for f in logs_dir.glob("*.json")])

data = []

for json_file in json_files:
    with open(json_file, "r") as f:
        log_data = json.load(f)

    # Extract key information from each log entry
    record = {
        "file_id": int(json_file.stem),
        "question": log_data.get("question", ""),
        "all_nodes": log_data.get("all_nodes", []),
        'conversations_as_string': log_data.get("conversations_as_string", ""),
        "starting_nodes_indices": log_data.get("starting_nodes_indices", []),
        "agent_answer_indices": log_data.get("agent_answer_indices", []),
        "answer_indices": log_data.get("answer_indices", []),
    }

    data.append(record)

df = pd.DataFrame(data).sort_values(by="file_id").reset_index(drop=True)

df["recall@all"] = df.apply(
    lambda row: len(set(row["answer_indices"]).intersection(set(row["agent_answer_indices"])))
    / len(set(row["answer_indices"])),
    axis=1,
)
df["hit@1"] = df.apply(
    lambda row: row["agent_answer_indices"][0] in row["answer_indices"] if row["agent_answer_indices"] else False,
    axis=1,
)
df["hit@5"] = df.apply(
    lambda row: len(set(row["answer_indices"]).intersection(set(row["agent_answer_indices"][:5]))) > 0,
    axis=1,
)
df["hit@10"] = df.apply(
    lambda row: len(set(row["answer_indices"]).intersection(set(row["agent_answer_indices"][:10]))) > 0,
    axis=1,
)
df["recall@20"] = df.apply(
    lambda row: len(set(row["answer_indices"]).intersection(set(row["agent_answer_indices"][:20])))
    / len(set(row["answer_indices"])),
    axis=1,
)

[
    ("n", len(df)),
    ("Hit@1", float(round(df["hit@1"].mean(), 3))),
    ("Hit@5", float(round(df["hit@5"].mean(), 3))),
    ("Recall@20", float(round(df["recall@20"].mean(), 3))),
    ("Recall@all", float(round(df["recall@all"].mean(), 3))),
]

[('n', 7),
 ('Hit@1', 0.429),
 ('Hit@5', 0.429),
 ('Recall@20', 0.429),
 ('Recall@all', 0.429)]

### When it went wrong, what happened?

In [38]:
df_wrong = df[df['recall@all'] == 0]

In [39]:
df_wrong['startding_node_names'] = df_wrong['starting_nodes_indices'].apply(
    lambda x: [graph.get_node_by_index(idx).name for idx in x]
)
df_wrong['answer_node_names'] = df_wrong['answer_indices'].apply(
    lambda x: [graph.get_node_by_index(idx).name for idx in x]
)   
df_wrong['agent_answer_node_names'] = df_wrong['agent_answer_indices'].apply(
    lambda x: [graph.get_node_by_index(idx).name for idx in x]
)   

/var/folders/rv/0brfn3g10137qh8fcq6ch2c00000gn/T/ipykernel_98658/420965864.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wrong['startding_node_names'] = df_wrong['starting_nodes_indices'].apply(
/var/folders/rv/0brfn3g10137qh8fcq6ch2c00000gn/T/ipykernel_98658/420965864.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wrong['answer_node_names'] = df_wrong['answer_indices'].apply(
/var/folders/rv/0brfn3g10137qh8fcq6ch2c00000gn/T/ipykernel_98658/420965864.py:7: SettingWithCopyWarning: 
A value i

In [40]:
for _, row in df_wrong.iterrows():
    print("Question:", row["question"])
    print("Starting node names:", row["startding_node_names"])
    print("Starting nodes indices:", row["starting_nodes_indices"])
    print("Answer node names:", row["answer_node_names"])
    print("Answer indices:", row["answer_indices"])
    print("Agent answer node names:", row["agent_answer_node_names"])
    print()

Question: Which publications from Fairleigh Dickinson University authors examine volatile entrapment findings in their studies?
Starting node names: ['Fairleigh Dickinson University', 'Entrapment', 'Volatile Transport inside Super-Earths by Entrapment in the Water Ice Matrix', 'Bubble entrapment through topological change']
Starting nodes indices: [1105606, 1140264, 1440747, 1611894]
Answer node names: ["Evidence for the sequestration of hydrogen-bearing volatiles towards the Moon's southern pole-facing slopes"]
Answer indices: [1396816]
Agent answer node names: ['The regime of large bubble entrapment during a single drop impact on a liquid pool', 'Numerical analysis on air entrapment during a droplet impacts on a dry flat surface', 'Bubble entrapment during sphere impact onto quiescent liquid surfaces', 'Bubbles entrapment for drops impinging on polymer surfaces: The roughness effect']

Question: Does any research from Ningbo University of Technology discuss the application of nanopar

In [26]:
graph.get_node_by_index(128417)

AuthorNode(name=James M. Burst, index=128417, type=author)

In [27]:
graph.get_khop_subgraph(graph.get_node_by_index(37182), 2).nodes_df

,id,mag_id,Rank,DisplayName,LastKnownAffiliationId,PaperCount,CitationCount,type,new_id,Level,...,ConferenceSeriesId.1,Location,StartDate,EndDate,ConferenceInstancesPaperCount,ConferenceInstanceCitationCount,title,abstract,index,name
63,65,1503591,17747.0,Priyamvada Jadaun,86519309.0,17.0,61.0,author,63,NaN,...,NaN,None,None,None,NaN,NaN,None,None,63,Priyamvada Jadaun
186,189,4129598,12713.0,Harry L. Swinney,86519309.0,430.0,21604.0,author,186,NaN,...,NaN,None,None,None,NaN,NaN,None,None,186,Harry L. Swinney
212,215,4837528,11418.0,Rodney S. Ruoff,48566637.0,659.0,124985.0,author,212,NaN,...,NaN,None,None,None,NaN,NaN,None,None,212,Rodney S. Ruoff
289,292,7433618,18317.0,Solomon Manukure,86519309.0,11.0,48.0,author,289,NaN,...,NaN,None,None,None,NaN,NaN,None,None,289,Solomon Manukure
329,332,8253814,14510.0,Roger T. Bonnecaze,86519309.0,167.0,4240.0,author,329,NaN,...,NaN,None,None,None,NaN,NaN,None,None,329,Roger T. Bonnecaze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863909,719026,2962876139,NaN,None,NaN,NaN,NaN,paper,1863909,NaN,...,-1.0,-1,-1,-1,-1.0,-1.0,The MASSIVE Survey. II. Stellar Population Tre...,We examine stellar population gradients in ?10...,1863909,The MASSIVE Survey. II. Stellar Population Tre...
1865184,721632,2963351266,NaN,None,NaN,NaN,NaN,paper,1865184,NaN,...,-1.0,-1,-1,-1,-1.0,-1.0,A simultaneous search for High-z LAEs and LBGs...,We have undertaken a comprehensive search for ...,1865184,A simultaneous search for High-z LAEs and LBGs...
1866634,724663,2963923661,NaN,None,NaN,NaN,NaN,paper,1866634,NaN,...,-1.0,-1,-1,-1,-1.0,-1.0,J-PLUS: analysis of the intracluster light in ...,Y. J-T. also acknowledges financial support fr...,1866634,J-PLUS: analysis of the intracluster light in ...
1868541,728033,2967384074,NaN,None,NaN,NaN,NaN,paper,1868541,NaN,...,-1.0,-1,-1,-1,-1.0,-1.0,The first maps of κd - the dust mass absorptio...,"The dust mass absorption coefficient, κd is th...",1868541,The first maps of κd - the dust mass absorptio...


In [28]:
graph.get_node_by_index(96612)

AuthorNode(name=M. Schneiter, index=96612, type=author)